In [1]:
from polyglot.text import Text

import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=7,progress_bar=True)

from parallelbar import progress_map

from utils import *

from nltk import ngrams
import nltk

import scipy
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
import xgboost as xgb

from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    recall_score,
    precision_score,
    roc_auc_score, confusion_matrix, roc_curve, classification_report
)

from sentence_transformers import SentenceTransformer

import random

from datasets import Dataset, DatasetDict
import pandas as pd
import torch

from transformers import AutoTokenizer

from transformers import BertAdapterModel, BertConfig

from transformers import PfeifferConfig

from transformers import TrainingArguments, AdapterTrainer, EvalPrediction
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

torch.cuda.is_available()

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/marek/anaconda3/envs/nlp/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


True

## POS CZ data

In [2]:
if False:
    df_cz_cz = pd.read_csv('../datasets/demagog_nlp_cz/converted-exp-CZ.tsv', sep='\t')

    df_cz_cz['text_clean'] = df_cz_cz['statementText'].apply(lambda x: clean_przyp(x))

    tasks = df_cz_cz['text_clean'].values.tolist()
    result = progress_map(extract_features_cz, tasks, n_cpu=7, chunk_size=1, core_progress=True)
    
    df_cz_cz['TEXT_POS'] = result

    df_cz_cz['TEXT_POS'] = df_cz_cz['TEXT_POS'].str.join(" ")
    
    df_cz_cz.to_csv('../datasets/ready2use/fake_news_features_cz_CZ.csv', sep=';', index=False, encoding='utf8')
else:
    df_cz_cz = pd.read_csv('../datasets/ready2use/fake_news_features_cz_CZ.csv', sep=';')

In [3]:
df_cz_cz.shape

(9082, 10)

### The same code for SK data

In [4]:
if False:
    df_cz_sk = pd.read_csv('../datasets/demagog_nlp_cz/converted-exp-SK.tsv', sep='\t')

    df_cz_sk['text_clean'] = df_cz_sk['statementText'].apply(lambda x: clean_przyp(x))

    tasks = df_cz_sk['text_clean'].values.tolist()
    result = progress_map(extract_features_cz, tasks, n_cpu=7, chunk_size=1, core_progress=True)
    
    df_cz_sk['TEXT_POS'] = result

    df_cz_sk['TEXT_POS'] = df_cz_sk['TEXT_POS'].str.join(" ")
    
    df_cz_sk.to_csv('../datasets/ready2use/fake_news_features_cz_SK.csv', sep=';', index=False, encoding='utf8')
else:
    df_cz_sk = pd.read_csv('../datasets/ready2use/fake_news_features_cz_SK.csv', sep=';')

In [5]:
df_cz_sk.shape

(12554, 10)

In [6]:
df_cz = pd.concat([df_cz_sk, df_cz_sk])
df_cz.shape

(25108, 10)

In [7]:
df_cz['statementState'] = df_cz['statementState'].str.strip()

df_cz = df_cz[ df_cz['statementState'] != 'MISLEADING' ]
df_cz = df_cz[ df_cz['statementState'] != 'UNVERIFIABLE' ]
df_cz = df_cz[ df_cz['statementState'] != 'null' ]

df_cz = df_cz.reset_index(drop=True)

df_cz['assestment'] = df_cz['statementState'].replace({
    'FALSE' : 0,
#     'Manipulacja' : 1,
    'TRUE' : 1
}).astype(int)

In [8]:
df_cz['assestment'].value_counts()

1    15974
0     3340
Name: assestment, dtype: int64

## POS ENG data

In [9]:
df_en = pd.read_csv('../datasets/politifact/politifact.csv', sep=',', index_col=0)

df_en.loc[:, 'fact'] = df_en['fact'].replace({
    'half-true' : 'true',
    'mostly-true' : 'true',
    'barely-true' : 'false',
    'pants-fire' : 'false',
})

df_en = df_en[df_en['fact'].isin(['true', 'false'])]

In [10]:
df_en.shape

(19151, 11)

In [11]:
df_en = df_en[['sources_quote', 'fact']]
df_en.columns  = [
    'statement',
    'label'
]

df_en['text_clean'] = df_en['statement'].apply(lambda x: clean_przyp(x))

df_en['assestment'] = df_en['label'].replace({
    'false' : 0,
#     'Manipulacja' : 1,
    'true' : 1
}).astype(int)

## Use EN/CZ data as training

In [12]:
df_all = pd.concat([df_en[['assestment', 'text_clean']], 
                    df_cz[['assestment', 'text_clean']]])

In [13]:
df_all = df_cz[['assestment', 'text_clean']]

In [14]:
df_all['assestment'].value_counts()

1    15974
0     3340
Name: assestment, dtype: int64

In [15]:
df_all['sentence'] = df_all['text_clean'].apply(lambda x: x.replace('\n',' ').strip())
df_all['labels'] = df_all['assestment'].astype(int)

df_all = df_all.sample(frac=1, random_state=111)

df_all.shape

/tmp/ipykernel_1266322/3004308602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['sentence'] = df_all['text_clean'].apply(lambda x: x.replace('\n',' ').strip())
/tmp/ipykernel_1266322/3004308602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['labels'] = df_all['assestment'].astype(int)


(19314, 4)

### Load pl data

In [16]:
df = pd.read_csv('../datasets/ready2use/text_celan_pl_dataset.csv',
                 index_col=0,
                 sep=';',header=None,names='sentence labels'.split())

df['sentence'] = df['sentence'].apply(lambda x: x.replace('\n',' ').strip())
df['labels'] = df['labels'].astype(int)

df = df.sample(frac=1, random_state=111)

df.shape

(6542, 2)

In [17]:
model_name = 'DeepPavlov/bert-base-bg-cs-pl-ru-cased'

In [18]:
tok = AutoTokenizer.from_pretrained(model_name)

In [19]:
labels = sorted(set(df_all['labels']))
num_labels = len(labels)
label2id = dict(zip(labels,range(num_labels)))
id2label = {v: k for k, v in label2id.items()}

def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    batch['labels'] = np.array(batch['labels'])
    return tok(text=batch['sentence'], max_length=64, truncation=True, padding="max_length")

In [20]:
config = BertConfig.from_pretrained(
    model_name,
    num_labels=num_labels
)

adapter_config = PfeifferConfig()


training_args = TrainingArguments(
    remove_unused_columns=False, # ensure the dataset labels are properly passed to the model
    
    fp16=True,
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
    label_smoothing_factor=0.0,
    weight_decay=0.05,
    warmup_steps=600,
    output_dir="./training_output",
    overwrite_output_dir=True,
    
#     logging_strategy="no"
    
    logging_steps=200,
    logging_first_step=False,
    logging_dir='./tb_logs',
    evaluation_strategy='steps',
    report_to='tensorboard',
    save_steps=200
)

def compute_accuracy_f1score(p: EvalPrediction):
    preds = [id2label[pred_id] for pred_id in np.argmax(p.predictions, axis=1)]
    target_labels = [id2label[label_id] for label_id in p.label_ids]
    return {"accuracy": accuracy_score(target_labels,preds),
           "f1score": f1_score(target_labels,preds)}

In [21]:
ds = DatasetDict()
ds['train'] = Dataset.from_pandas(df_all)
ds['valid'] = Dataset.from_pandas(df)


ds = ds.map(encode_batch, batched=True, batch_size=len(ds['train']))
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


model = BertAdapterModel.from_pretrained(model_name, config=config).to('cuda')

adapter_name='fake_news_multilang_0'

model.add_adapter(adapter_name,config=adapter_config)
model.add_classification_head(
    adapter_name,
    id2label=id2label,
    num_labels=num_labels
  )

model.train_adapter(adapter_name)


trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["valid"],
    compute_metrics=compute_accuracy_f1score,
)

trainer.train()

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at DeepPavlov/bert-base-bg-cs-pl-ru-cased were not used when initializing BertAdapterModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using cuda_amp half precision backend
/home/marek/anaconda3/envs/nlp/lib/python3.

Step,Training Loss,Validation Loss,Accuracy,F1score
200,0.562400,0.861352,0.474320,0.643442
400,0.465900,1.057838,0.474320,0.643442
600,0.473700,1.098155,0.474320,0.643442
800,0.464100,1.000463,0.474320,0.643442
1000,0.456500,1.116927,0.474320,0.643442
1200,0.445000,0.993393,0.474473,0.643509
1400,0.457700,1.007912,0.474320,0.643442
1600,0.453400,1.161575,0.474473,0.643509
1800,0.445500,1.250108,0.474473,0.643509
2000,0.473900,1.099069,0.474320,0.643442


***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-200
Configuration saved in ./training_output/checkpoint-200/fake_news_multilang_0/adapter_config.json
Module weights saved in ./training_output/checkpoint-200/fake_news_multilang_0/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-200/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-200/fake_news_multilang_0/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-200/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-200/fake_news_multilang_0/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-400
Configuration saved in ./training_output/checkpoint-400/fake_news_multilang_0/adapter_config.json
Module weights saved in ./training_output/checkpo

...

Module weights saved in ./training_output/checkpoint-15800/fake_news_multilang_0/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-15800/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-15800/fake_news_multilang_0/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-16000
Configuration saved in ./training_output/checkpoint-16000/fake_news_multilang_0/adapter_config.json
Module weights saved in ./training_output/checkpoint-16000/fake_news_multilang_0/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-16000/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-16000/fake_news_multilang_0/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-16000/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-16000/fake

Configuration saved in ./training_output/checkpoint-18000/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-18000/fake_news_multilang_0/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-18200
Configuration saved in ./training_output/checkpoint-18200/fake_news_multilang_0/adapter_config.json
Module weights saved in ./training_output/checkpoint-18200/fake_news_multilang_0/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-18200/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-18200/fake_news_multilang_0/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-18200/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-18200/fake_news_multilang_0/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 6542
  Batch si

Module weights saved in ./training_output/checkpoint-20200/fake_news_multilang_0/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-20400
Configuration saved in ./training_output/checkpoint-20400/fake_news_multilang_0/adapter_config.json
Module weights saved in ./training_output/checkpoint-20400/fake_news_multilang_0/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-20400/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-20400/fake_news_multilang_0/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-20400/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-20400/fake_news_multilang_0/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-20600
Configuration saved in ./tr

***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-22600
Configuration saved in ./training_output/checkpoint-22600/fake_news_multilang_0/adapter_config.json
Module weights saved in ./training_output/checkpoint-22600/fake_news_multilang_0/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-22600/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-22600/fake_news_multilang_0/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-22600/fake_news_multilang_0/head_config.json
Module weights saved in ./training_output/checkpoint-22600/fake_news_multilang_0/pytorch_model_head.bin
***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32
Saving model checkpoint to ./training_output/checkpoint-22800
Configuration saved in ./training_output/checkpoint-22800/fake_news_multilang_0/adapter_config.json
Module weights saved in ./train

TrainOutput(global_step=24150, training_loss=0.3441591379706657, metrics={'train_runtime': 2745.6658, 'train_samples_per_second': 70.344, 'train_steps_per_second': 8.796, 'total_flos': 6462303707520000.0, 'train_loss': 0.3441591379706657, 'epoch': 10.0})

In [22]:
eval_out = trainer.evaluate()

print('Acc', eval_out['eval_accuracy'])
print('F1', eval_out['eval_f1score'])

***** Running Evaluation *****
  Num examples = 6542
  Batch size = 32


Acc 0.4544481809844084
F1 0.6027824151363383
